## Compute SPI for diffusion model evaluation

In [64]:
import glob
import hdc.algo
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xss

from scipy.stats import spearmanr, pearsonr
from read_data import get_cond_ecmwf_dataset, get_mean_std_data

**Read forecasts and chirps** 

In [76]:
train_ecmwf = xr.open_mfdataset('data/train/*.zarr').ecmwf.load()
test_ecmwf = xr.open_mfdataset('data/test/*.zarr').ecmwf.load()
ecmwf = xr.concat([train_ecmwf, test_ecmwf], dim='time')

train_chirps = xr.open_mfdataset('data/train/*.zarr').chirps.load()
test_chirps = xr.open_mfdataset('data/test/*.zarr').chirps.load()
chirps = xr.concat([train_chirps, test_chirps], dim='time')

/envs/user/swirl/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/envs/user/swirl/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/envs/user/swirl/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/envs/user/swirl/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/envs/user/swirl/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/envs/user/swirl/lib/python3.11/site-pa

**Read benchmarks** 

In [66]:
train_qm = xr.open_zarr('data/benchmarks_results/QM/train_quantile_mapping.zarr').scen.load()
test_qm = xr.open_zarr('data/benchmarks_results/QM/test_quantile_mapping.zarr').scen.load()
qm = xr.concat([train_qm, test_qm], dim='time')

train_bilint = xr.open_zarr('data/benchmarks_results/bilinear/train_bilinear.zarr').tp.load()
test_bilint = xr.open_zarr('data/benchmarks_results/bilinear/test_bilinear.zarr').bilint.load()
bilint = xr.concat([train_bilint, test_bilint], dim='time')

#train_climax = xr.open_zarr('data/benchmarks_results/ClimaX/train_climax.zarr')
#test_climax = xr.open_zarr('data/benchmarks_results/ClimaX/test_climax.zarr')
#climax = xr.concat([train_climax, test_climax], dim='time')

**Compute SPI** 

Diffusion

In [67]:
train_diff_files = glob.glob('data/diffusion/batch_train_*.npy')
train_diff_samples = np.concatenate([np.load(file) for file in train_diff_files], axis=0)
train_diff = np.mean(train_diff_samples, axis=1)[:, :, :, 0]
train_diff_da = xr.DataArray(
    data=train_diff,
    dims=["time", "latitude", "longitude"],
    coords=train_ecmwf.coords,
)

In [68]:
test_diff_samples = np.load('data/diffusion/batch_5samples.npy')
test_diff = np.mean(test_diff_samples, axis=1)[:, :, :, 0]
test_diff_da = xr.DataArray(
    data=test_diff,
    dims=["time", "latitude", "longitude"],
    coords=test_ecmwf.coords,
)

In [69]:
### Get mean and var for de-normalizing
mean_chirps, var_chirps, mean_ecmwf, var_ecmwf = get_mean_std_data(split="train")

In [70]:
train_diff_da = train_diff_da * (var_chirps + 1e-4) + mean_chirps
test_diff_da = test_diff_da * (var_chirps + 1e-4) + mean_chirps

In [49]:
da_diff = xr.concat([train_diff_da, test_diff_da], dim='time')
da_diff.attrs['nodata'] = np.nan

In [50]:
train_ecmwf.time.isel(time=-1).values

numpy.datetime64('2015-12-21T00:00:00.000000000')

In [51]:
spi_diffusion = da_diff.hdc.algo.spi(calibration_start=train_ecmwf.time.isel(time=0).values, calibration_stop=train_ecmwf.time.isel(time=-1).values)

In [55]:
spi_diffusion = spi_diffusion.sel(time=test_ecmwf.time)

In [85]:
spi_diffusion

<xarray.DataArray (latitude: 28, longitude: 28, time: 168)> Size: 263kB
array([[[ -813,  -108,  6007, ...,  -381,  2457,  1541],
        [ -741,   322,  6780, ...,  -438,  2779,  1812],
        [ -222,   522,  7188, ...,  -111,  2982,  1776],
        ...,
        [  308,  -659,   127, ...,  1037,  1041, -1917],
        [  263,  -539,  -162, ...,   726,  1621, -1908],
        [  -80,  -681,  -102, ...,   720,  1800, -1600]],

       [[ -630,  -226,  3440, ...,  -285,  2461,  1677],
        [ -947,   -13,  6708, ...,  -273,  2636,  1941],
        [ -172,   281,  3781, ...,    40,  2572,  2143],
        ...,
        [  397, -1132,  1239, ...,   716,  2899, -2002],
        [  284,  -718,  2167, ...,   371,  2778, -1822],
        [   48,  -925,  1519, ...,   426,  2676, -1650]],

       [[  -84,   240,  3833, ...,   292,  2850,  1727],
        [ -173,   251,  3819, ...,   235,  3130,  2000],
        [ -275,   575,  4110, ...,   338,  2823,  2122],
        ...,
...
        ...,
        [-1483,  -846, -4510, ...,  -900, -3048,   733],
        [ -876,  -404, -4438, ...,  -676, -1480,   714],
        [-1015,  -119,   -64, ...,  -579, -2828,   856]],

       [[  327,  -280,  -952, ...,  -208, -4824,  1194],
        [  218,  -296,  -928, ...,     3, -3155,  1061],
        [  139,  -291, -1229, ...,   -23, -2398,  1002],
        ...,
        [-1398, -1182, -1016, ..., -1306,   357,  -308],
        [ -879,  -657, -1340, ..., -1025,  1129,   145],
        [ -949,  -329, -1428, ...,  -807, -2077,   129]],

       [[ -196,    61,  -811, ...,    70, -4660,  1528],
        [  311,   -98, -5158, ...,    80, -4983,  1393],
        [  285,  -100, -1676, ...,   101, -4666,  1507],
        ...,
        [ -898, -1232, -4957, ..., -1248,   764,  -687],
        [-1546,  -916, -1844, ..., -1030,   168,  -590],
        [-1191,  -505, -1326, ...,  -720, -2808,   140]]], dtype=int16)
Coordinates:
    ensemble     int64 8B 0
  * latitude     (latitude) float64 224B -15.13 -15.18 -15.23 ... -16.43 -16.48
  * longitude    (longitude) float64 224B 33.12 33.17 33.23 ... 34.42 34.48
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 1kB 2016-01-01 2016-01-11 ... 2023-12-21
Attributes:
    nodata:                 nan
    spi_calibration_start:  1981-01-01
    spi_calibration_stop:   2015-12-21

In [87]:
spi_diffusion.to_zarr("data/diffusion/test_diffusion_spi.zarr")

Benchmarks

In [60]:
bilint.attrs['nodata'] = np.nan
spi_bilint = bilint.hdc.algo.spi(calibration_start=train_ecmwf.time.isel(time=0).values, calibration_stop=train_ecmwf.time.isel(time=-1).values)

In [88]:
spi_bilint = spi_bilint.sel(time=test_ecmwf.time)

In [89]:
spi_bilint.to_zarr("data/benchmarks_results/bilinear/test_bilint_spi.zarr")

In [62]:
qm.attrs['nodata'] = np.nan
spi_qm = qm.hdc.algo.spi(calibration_start=train_ecmwf.time.isel(time=0).values, calibration_stop=train_ecmwf.time.isel(time=-1).values)

In [80]:
spi_qm = spi_qm.sel(time=test_ecmwf.time)

In [81]:
spi_qm.to_zarr("data/benchmarks_results/QM/test_qm_spi.zarr")

Ground truth

In [78]:
spi_gt = chirps.hdc.algo.spi(calibration_start=train_ecmwf.time.isel(time=0).values, calibration_stop=train_ecmwf.time.isel(time=-1).values)

In [82]:
spi_gt = spi_gt.sel(time=test_ecmwf.time)

In [84]:
spi_gt.to_zarr("data/test_spi_chirps.zarr")